In the following section, we are searching for the optimal hyperparameters needed in each model. Therefore, we are using the Grid-Search-Method with cross-validation which we get from the package sklearn. We reduce the number of Parameters and use default Parameters for some models in order to obtain better executability.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import os 
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras import regularizers

import warnings
warnings.filterwarnings("ignore")

In [19]:
%matplotlib inline
df = pd.read_excel(r'\path\to\DSML_Predictors_final.xlsx', sheet_name = 'Predictors (täglich)',header = 4, usecols="A:AT")
df.head()

,DATE,USTB3Md_t,Zt,Class_p,WILL_t,RABS_WIL_t,R_WIL_t,SQR_WIL_t,DAX_t,R_DAX_t,...,UR_t,R_UR_t,USGDP_t,R_USGDP_t,VIX_t,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,1991-01-01 00:00:00,0.017722,-0.017722,1.0,3101.36,0.000000,0.000000,0.000000,1398.23,0.000000,...,6.4,0.0,9275.276,0.0,26.38,NaN,NaN,NaN,NaN,NaN
1,1991-01-02 00:00:00,0.017722,-0.022195,1.0,3069.58,-0.010247,-0.004473,0.000020,1366.10,-0.010096,...,6.4,0.0,9275.276,0.0,26.62,NaN,NaN,NaN,NaN,NaN
2,1991-01-03 00:00:00,0.017722,-0.022967,1.0,3032.73,-0.012005,-0.005245,0.000028,1366.65,0.000175,...,6.4,0.0,9275.276,0.0,27.93,NaN,NaN,NaN,NaN,NaN
3,1991-01-04 00:00:00,0.017722,-0.018856,1.0,3024.82,-0.002608,-0.001134,0.000001,1396.07,0.009250,...,6.4,0.0,9275.276,0.0,27.19,NaN,NaN,NaN,NaN,NaN
4,1991-01-07 00:00:00,0.017722,-0.025084,1.0,2973.98,-0.016808,-0.007361,0.000054,1358.16,-0.011956,...,6.4,0.0,9275.276,0.0,28.95,NaN,NaN,NaN,NaN,NaN


In [20]:
"""Find rows with missing value and delete them"""
delete_rows = []
for i in range(0, len(df)):
    for j in range(1,1):
        if(np.isnan(df.iloc[i][j]) or df.iloc[i][j] == 0):
            delete_rows.append(i)
            break  
#print("Zeilen mit folgenden Indices werden gelöscht:", delete_rows)
df.drop(labels=delete_rows, axis=0, inplace = True)
df.reset_index(drop = True, inplace = True)

In [21]:
"""Dataframe mit relevanten Spalten erstellen"""
df_final = df[['DATE', 'Class_p', 'RABS_WIL_t','R_WIL_t','SQR_WIL_t','R_DAX_t','R_FTSE_t','R_USDUK_t','R_COP_t', 'R_USD_t', 'R_S&P_t', 'R_USPYC1Y_t', 'R_USPYC10Y_t', 'R_USPYC30Y_t', 'R_LIB3M_t', 'R_USTB3M_t', 'R_INFL_t', 'R_USDJ_t', 'R_USFFTR_t', 'R_UR_t', 'R_USGDP_t', 'VIX_t']]
df_final['R_WIL_t'] = df_final['R_WIL_t'].astype(float)
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)
df_final.dropna(inplace=True)
df_final.loc[:,'Class'] = 2.0
print(df_final.describe())

           Class_p   RABS_WIL_t      R_WIL_t     SQR_WIL_t      R_DAX_t  \
count  8044.000000  8044.000000  8044.000000  8.044000e+03  8044.000000   
mean      1.439334     0.000403     0.000148  2.385927e-05     0.000131   
std       0.570678     0.011220     0.004883  8.997339e-05     0.005999   
min       1.000000    -0.122876    -0.056939  0.000000e+00    -0.056697   
25%       1.000000    -0.003936    -0.001713  6.461532e-07    -0.002499   
50%       1.000000     0.000400     0.000174  4.157272e-06     0.000214   
75%       2.000000     0.005287     0.002290  1.802552e-05     0.003091   
max       3.000000     0.113758     0.046791  3.242064e-03     0.046893   

          R_FTSE_t    R_USDUK_t      R_COP_t      R_USD_t      R_S&P_t  ...  \
count  8044.000000  8044.000000  8044.000000  8044.000000  8044.000000  ...   
mean      0.000066    -0.000018     0.000072     0.000007     0.000142  ...   
std       0.004760     0.002543     0.006757     0.002153     0.004891  ...   
min     

In [22]:
df_final.head()

,DATE,Class_p,RABS_WIL_t,R_WIL_t,SQR_WIL_t,R_DAX_t,R_FTSE_t,R_USDUK_t,R_COP_t,R_USD_t,...,R_USPYC30Y_t,R_LIB3M_t,R_USTB3M_t,R_INFL_t,R_USDJ_t,R_USFFTR_t,R_UR_t,R_USGDP_t,VIX_t,Class
0,1991-01-01 00:00:00,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,26.38,2.0
1,1991-01-02 00:00:00,1.0,-0.010247,-0.004473,0.000020,-0.010096,-0.003091,0.002804,0.009793,-0.001466,...,-0.1200,-0.01563,0.0,0.0,0.0,0.0,0.0,0.0,26.62,2.0
2,1991-01-03 00:00:00,1.0,-0.012005,-0.005245,0.000028,0.000175,-0.002148,0.000893,-0.002137,0.000105,...,-0.0302,-0.18750,0.0,0.0,0.0,0.0,0.0,0.0,27.93,2.0
3,1991-01-04 00:00:00,1.0,-0.002608,-0.001134,0.000001,0.009250,0.001699,-0.002910,-0.009739,0.002875,...,0.1084,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,27.19,2.0
4,1991-01-07 00:00:00,1.0,-0.016808,-0.007361,0.000054,-0.011956,-0.002623,-0.006221,0.003804,0.006772,...,0.0966,0.20313,0.0,0.0,0.0,0.0,0.0,0.0,28.95,2.0


In [23]:
#Lags für Return WIL
for i in [1,2,3,4,5,6,7,8,9,10]:
    df_final['R_WIL_t-'+str(i)] = df['R_WIL_t'].shift(i)
    df_final['SQR_WIL_t-'+str(i)] = df['SQR_WIL_t'].shift(i)
    df_final['R_DAX_t-'+str(i)] = df['R_DAX_t'].shift(i)
    df_final['R_FTSE_t-'+str(i)] = df['R_FTSE_t'].shift(i)
    df_final['R_USDUK_t-'+str(i)] = df['R_USDUK_t'].shift(i)
    df_final['R_COP_t-'+str(i)] = df['R_COP_t'].shift(i)
    df_final['R_USD_t-'+str(i)] = df['R_USD_t'].shift(i)
    df_final['R_S&P_t-'+str(i)] = df['R_S&P_t'].shift(i)
    df_final['R_USPYC1Y_t-'+str(i)] = df['R_USPYC1Y_t'].shift(i)
    df_final['R_USPYC10Y_t-'+str(i)] = df['R_USPYC10Y_t'].shift(i)
    df_final['R_USPYC30Y_t-'+str(i)] = df['R_USPYC30Y_t'].shift(i)
    df_final['R_LIB3M_t-'+str(i)] = df['R_LIB3M_t'].shift(i)
    df_final['R_USTB3M_t-'+str(i)] = df['R_USTB3M_t'].shift(i)
    df_final['R_INFL_t-'+str(i)] = df['R_INFL_t'].shift(i)
    df_final['R_USDJ_t-'+str(i)] = df['R_USDJ_t'].shift(i)
    df_final['R_USFFTR_t-'+str(i)] = df['R_USFFTR_t'].shift(i)
    df_final['R_UR_t-'+str(i)] = df['R_UR_t'].shift(i)
    df_final['R_USGDP_t-'+str(i)] = df['R_USGDP_t'].shift(i)
    df_final['VIX_t-'+str(i)] = df['VIX_t'].shift(i)
    

In [24]:
df_final.dropna(inplace=True)
df_final.reset_index(drop = True, inplace = True)

In [25]:
def getSubDataframe(dataframe, end_year):
    
    df_sub = pd.DataFrame(columns=list(dataframe))
    df_sub_train = pd.DataFrame(columns=list(dataframe))
    df_sub_test = pd.DataFrame(columns=list(dataframe))
    for i in range(0, len(dataframe)):
        
        year = dataframe.loc[i,'DATE'].year
        
        if year <= end_year:
            
            df_sub = df_sub.append(dataframe.loc[i,:])
            
            if year <= (end_year - 2):
                
                df_sub_train = df_sub_train.append(dataframe.loc[i,:])
                
            if (year > (end_year -2)) and (year <= end_year):
                
                df_sub_test = df_sub_test.append(dataframe.loc[i,:])
        
    return df_sub, df_sub_train, df_sub_test

In [26]:
Perioden = [2009,2011,2013,2015,2017,2019,2021]
print(Perioden)

[2009, 2011, 2013, 2015, 2017, 2019, 2021]


In [ ]:
#Erstellen der Dataframes für jede Periode
for i in Perioden:
    globals()[f"df_periode_{i}"],globals()[f"df_train_periode_{i}"],globals()[f"df_test_periode_{i}"] = getSubDataframe(df_final, i)

In [29]:
for i in Perioden:
    globals()[f"df_test_periode_{i}"].reset_index(drop = True, inplace=True)

In [30]:
df_test_periode_2021

,DATE,Class_p,RABS_WIL_t,R_WIL_t,SQR_WIL_t,R_DAX_t,R_FTSE_t,R_USDUK_t,R_COP_t,R_USD_t,...,R_USPYC10Y_t-10,R_USPYC30Y_t-10,R_LIB3M_t-10,R_USTB3M_t-10,R_INFL_t-10,R_USDJ_t-10,R_USFFTR_t-10,R_UR_t-10,R_USGDP_t-10,VIX_t-10
0,2020-01-01,1.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0398,0.0369,0.00550,0.0,0.0,0.0,0.0,0.0,0.0,12.58
1,2020-01-02,2.0,0.007750,0.003353,1.124130e-05,0.004465,0.003547,-0.001971,0.001023,0.002068,...,-0.0031,0.0144,0.01975,0.0,0.0,0.0,0.0,0.0,0.0,12.50
2,2020-01-03,1.0,-0.006221,-0.002710,7.344346e-06,-0.005445,0.001032,-0.003854,-0.004213,-0.000045,...,0.0040,-0.0113,0.00700,0.0,0.0,0.0,0.0,0.0,0.0,12.51
3,2020-01-06,2.0,0.003357,0.001455,2.118374e-06,-0.003038,-0.002690,0.002947,0.000746,-0.000763,...,0.0033,-0.0045,0.01188,0.0,0.0,0.0,0.0,0.0,0.0,12.61
4,2020-01-07,1.0,-0.002484,-0.001080,1.166958e-06,0.003291,-0.000085,-0.001306,0.000922,0.001480,...,-0.0183,-0.0022,0.01387,0.0,0.0,0.0,0.0,0.0,0.0,12.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,2021-10-25,2.0,0.005278,0.002286,5.225709e-06,0.001569,0.001100,-0.000095,0.009317,0.000788,...,0.0000,0.0000,0.00062,0.0,0.0,0.0,0.0,0.0,0.0,20.00
474,2021-10-26,2.0,0.000754,0.000327,1.072123e-07,0.004372,0.003283,0.000678,-0.004999,0.000648,...,-0.0296,-0.0562,0.00500,0.0,0.0,0.0,0.0,0.0,0.0,19.85
475,2021-10-27,2.0,-0.007019,-0.003059,9.357940e-06,-0.001415,-0.001456,-0.002463,-0.012771,-0.000694,...,-0.0295,-0.0601,-0.00300,0.0,0.0,0.0,0.0,0.0,0.0,18.64
476,2021-10-28,2.0,0.011038,0.004767,2.272747e-05,-0.000262,-0.000228,0.002715,0.008387,-0.002089,...,-0.0347,-0.0231,-0.00150,0.0,0.0,0.0,0.0,0.0,0.0,16.86


In [31]:
for i in Perioden:
    globals()[f"quantile_25_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.25)
    globals()[f"quantile_75_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.75)
    globals()[f"quantile_33_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.33)
    globals()[f"quantile_67_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.67)
    globals()[f"quantile_20_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.20)
    globals()[f"quantile_80_periode_{i}"] = globals()[f"df_train_periode_{i}"]['RABS_WIL_t'].quantile(.80)
    

In [32]:
for i in Perioden:
    for j in range(0, len(globals()[f"df_periode_{i}"])):
        if globals()[f"df_periode_{i}"].loc[j,'RABS_WIL_t'] <= globals()[f"quantile_25_periode_{i}"]:
            globals()[f"df_periode_{i}"].loc[j, 'Class'] = 1.0
        elif globals()[f"df_periode_{i}"].loc[j,'RABS_WIL_t'] > globals()[f"quantile_75_periode_{i}"]:
            globals()[f"df_periode_{i}"].loc[j,'Class'] = 3.0
        else:
            globals()[f"df_periode_{i}"].loc[j,'Class'] = 2.0
            
for i in Perioden:
    for j in range(0, len(globals()[f"df_train_periode_{i}"])):
        if globals()[f"df_train_periode_{i}"].loc[j,'RABS_WIL_t'] <= globals()[f"quantile_25_periode_{i}"]:
            globals()[f"df_train_periode_{i}"].loc[j, 'Class'] = 1.0
        elif globals()[f"df_train_periode_{i}"].loc[j,'RABS_WIL_t'] > globals()[f"quantile_75_periode_{i}"]:
            globals()[f"df_train_periode_{i}"].loc[j,'Class'] = 3.0
        else:
            globals()[f"df_train_periode_{i}"].loc[j,'Class'] = 2.0

for i in Perioden:
    for j in range(0, len(globals()[f"df_test_periode_{i}"])):
        if (globals()[f"df_test_periode_{i}"].loc[j,'RABS_WIL_t'] <= globals()[f"quantile_25_periode_{i}"]):
            globals()[f"df_test_periode_{i}"].loc[j, 'Class'] = 1.0
        elif globals()[f"df_test_periode_{i}"].loc[j,'RABS_WIL_t'] > globals()[f"quantile_75_periode_{i}"]:
            globals()[f"df_test_periode_{i}"].loc[j,'Class'] = 3.0
        else:
            globals()[f"df_test_periode_{i}"].loc[j,'Class'] = 2.0

In [33]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils



for i in Perioden:
    # Train and test data
    globals()[f"x_periode_{i}"] = globals()[f"df_periode_{i}"].drop(['DATE', 'Class', 'Class_p', 'RABS_WIL_t','R_WIL_t','SQR_WIL_t','R_DAX_t','R_FTSE_t','R_USDUK_t','R_COP_t', 'R_USD_t', 'R_S&P_t', 'R_USPYC1Y_t', 'R_USPYC10Y_t', 'R_USPYC30Y_t', 'R_LIB3M_t', 'R_USTB3M_t', 'R_INFL_t', 'R_USDJ_t', 'R_USFFTR_t', 'R_UR_t', 'R_USGDP_t', 'VIX_t'], axis=1)
    globals()[f"y_periode_{i}"] = globals()[f"df_periode_{i}"]['Class']
    globals()[f"x_train_periode_{i}"] = globals()[f"df_train_periode_{i}"].drop(['DATE', 'Class', 'Class_p', 'RABS_WIL_t','R_WIL_t','SQR_WIL_t','R_DAX_t','R_FTSE_t','R_USDUK_t','R_COP_t', 'R_USD_t', 'R_S&P_t', 'R_USPYC1Y_t', 'R_USPYC10Y_t', 'R_USPYC30Y_t', 'R_LIB3M_t', 'R_USTB3M_t', 'R_INFL_t', 'R_USDJ_t', 'R_USFFTR_t', 'R_UR_t', 'R_USGDP_t', 'VIX_t'], axis=1)
    globals()[f"x_test_periode_{i}"] = globals()[f"df_test_periode_{i}"].drop(['DATE', 'Class', 'Class_p', 'RABS_WIL_t','R_WIL_t','SQR_WIL_t','R_DAX_t','R_FTSE_t','R_USDUK_t','R_COP_t', 'R_USD_t', 'R_S&P_t', 'R_USPYC1Y_t', 'R_USPYC10Y_t', 'R_USPYC30Y_t', 'R_LIB3M_t', 'R_USTB3M_t', 'R_INFL_t', 'R_USDJ_t', 'R_USFFTR_t', 'R_UR_t', 'R_USGDP_t', 'VIX_t'], axis=1)
    globals()[f"y_train_periode_{i}"] = globals()[f"df_train_periode_{i}"]['Class']
    globals()[f"y_test_periode_{i}"] = globals()[f"df_test_periode_{i}"]['Class']

    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(globals()[f"y_periode_{i}"])
    globals()[f"encoded_y_periode_{i}"] = encoder.transform(globals()[f"y_periode_{i}"])
    # convert integers to dummy variables (i.e. one hot encoded)
    globals()[f"dummy_y_periode_{i}"] = np_utils.to_categorical(globals()[f"encoded_y_periode_{i}"])
    
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(globals()[f"y_train_periode_{i}"])
    globals()[f"encoded_y_train_periode_{i}"] = encoder.transform(globals()[f"y_train_periode_{i}"])
    # convert integers to dummy variables (i.e. one hot encoded)
    globals()[f"dummy_y_train_periode_{i}"] = np_utils.to_categorical(globals()[f"encoded_y_train_periode_{i}"])

    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(globals()[f"y_test_periode_{i}"])
    globals()[f"encoded_y_test_periode_{i}"] = encoder.transform(globals()[f"y_test_periode_{i}"])
    # convert integers to dummy variables (i.e. one hot encoded)
    globals()[f"dummy_y_test_periode_{i}"] = np_utils.to_categorical(globals()[f"encoded_y_test_periode_{i}"])




In [34]:
### scale the variables: (noch machen)
from sklearn.preprocessing import StandardScaler
SC= StandardScaler().fit(globals()[f"x_train_periode_{i}"][list(globals()[f"x_train_periode_{i}"])])
globals()[f"x_train_periode_{i}"][list(globals()[f"x_train_periode_{i}"])]=SC.transform(globals()[f"x_train_periode_{i}"][list(globals()[f"x_train_periode_{i}"])])
globals()[f"x_test_periode_{i}"][list(globals()[f"x_train_periode_{i}"])]=SC.transform(globals()[f"x_test_periode_{i}"][list(globals()[f"x_train_periode_{i}"])])


# Hyperparameter Search

In order to get the optimal hyperparameters for each period for each model, we use a Grid Search Method. Therefore, we need to define the hyperparameters on a particualr range. Due to a high time required for computing, we used the default values from the Paper Nevasalmi(2020). Especially for GBM we needed to use a littel amount of possible hyperparameters.

Definition of the five model estimators:

In [49]:
 estimator_KNN = KNeighborsClassifier()
 estimator_GBM = GradientBoostingClassifier()      
 estimator_SVM = svm.SVC()
 estimator_NN = Sequential()
 estimator_RF = RandomForestClassifier()

Definition of the parameters for K-Nearest Neighbors:

In [36]:
 grid_params_KNN = {
     'n_neighbors': [250,331,400],
     'leaf_size': [20, 30, 40],
     'weights': ['uniform', 'distance'],
     'metric': ['euclidean', 'manhattan','minkowski']
     }

Definition of the parameters for Gradient Boosting:

In [48]:
 grid_params_GBM = {
         'n_estimators': [50,100],
         'max_depth':[2,3],
         'min_samples_split':[2,4],
         'learning_rate':[0.005,0.001],
         'n_iter_no_change' : [800,931]
         }

Definition of the parameters for Support Vector Machine:

In [51]:
grid_params_SVM = {
     
     'kernel' : ['poly'], # default kernel='rbf'                                                
     'degree' : [3] , #default=3
     'gamma'  : ['scale'], #default='scale'   
     'C' : [0.1, 0.3, 0.5, 1.0] #default=1.0  
    }


Definition of the parameters for Neural Networks:

In [39]:
grid_params_NN = {
    'neuron_hidden' : [5,10,20],
    'activation_hidden' : ['sigmoid', 'tanh'],
    'optimizer' : ['adam', 'sgd'],
    'dropout' : [0.1,0.3],
    'weight_decay' : [0.1,0.3,0.6]
     }

Definition of the parameters for Random Forest:

In [46]:
 grid_params_RF= {
     'n_estimators': [100,300,500],
     'max_features': ['auto', 'log2'],
     'min_samples_leaf': [1, 2, 3]
     }

Visualization of the results of the Grid-Search-Method for K-Nearest Neighbor:

In [42]:
for i in Perioden:

    gs = GridSearchCV(
         estimator = estimator_KNN,
         param_grid = grid_params_KNN,
         verbose = 1,
         cv = 5,
         n_jobs = -1,
         scoring = 'accuracy'
         )
    #print("Periode " + str(i))
    #globals()[f"gs_results_Periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])
    #print(globals()[f"gs_results_Periode_{i}"].best_estimator_)
    
    globals()[f"grid_result_KNN_periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])

    # summarize results
    print("Periode " + str(i))
    print("Best: %f using %s" % (globals()[f"grid_result_KNN_periode_{i}"].best_score_, 
                                 globals()[f"grid_result_KNN_periode_{i}"].best_params_))
    globals()[f"means_periode_{i}"] = globals()[f"grid_result_KNN_periode_{i}"].cv_results_['mean_test_score']
    globals()[f"stds_periode_{i}"] = globals()[f"grid_result_KNN_periode_{i}"].cv_results_['std_test_score']
    globals()[f"params_periode_{i}"] = globals()[f"grid_result_KNN_periode_{i}"].cv_results_['params']
    for mean, stdev, param in zip(globals()[f"means_periode_{i}"], globals()[f"stds_periode_{i}"], globals()[f"params_periode_{i}"]):
        print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Periode 2009
Best: 0.479774 using {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'uniform'}
0.479774 (0.049828) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'uniform'}
0.477966 (0.055138) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'distance'}
0.475932 (0.049302) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'uniform'}
0.473898 (0.054292) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'distance'}
0.478644 (0.045150) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'uniform'}
0.475932 (0.047755) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'distance'}
0.475254 (0.051163) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'uniform'}
0.475480 (0.053418) with: {'leaf_size': 20, 'metric': '

Periode 2013
Best: 0.473768 using {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 400, 'weights': 'uniform'}
0.469013 (0.071738) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'uniform'}
0.465903 (0.072798) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'distance'}
0.469743 (0.067016) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'uniform'}
0.468646 (0.069959) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'distance'}
0.470476 (0.068139) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'uniform'}
0.470109 (0.067502) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'distance'}
0.469014 (0.074512) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'uniform'}
0.468647 (0.075325) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'distance'}
0.46919

Periode 2017
Best: 0.493943 using {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'uniform'}
0.491947 (0.040359) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'uniform'}
0.492869 (0.041370) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'distance'}
0.493943 (0.039783) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'uniform'}
0.491181 (0.043598) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'distance'}
0.491641 (0.041494) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'uniform'}
0.492563 (0.044319) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'distance'}
0.490565 (0.039654) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'uniform'}
0.488569 (0.042028) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'distance'}
0.49394

Periode 2021
Best: 0.503177 using {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'distance'}
0.502912 (0.004432) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'uniform'}
0.503044 (0.005304) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 250, 'weights': 'distance'}
0.502250 (0.003734) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'uniform'}
0.502250 (0.003734) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 331, 'weights': 'distance'}
0.501324 (0.002195) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'uniform'}
0.501853 (0.003240) with: {'leaf_size': 20, 'metric': 'euclidean', 'n_neighbors': 400, 'weights': 'distance'}
0.502912 (0.004432) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'uniform'}
0.503177 (0.004951) with: {'leaf_size': 20, 'metric': 'manhattan', 'n_neighbors': 250, 'weights': 'distance'}
0.5007

Visualization of the results of the Grid-Search-Method for Gradient Boosting:

In [ ]:
for i in Perioden:     
    gs = GridSearchCV(
         estimator = estimator_GBM,
         param_grid = grid_params_GBM,
         verbose = 1,
         cv = 5,
         n_jobs = -1,
         scoring = 'accuracy'
         )

    #print("Periode " + str(i))
    #globals()[f"gs_results_Periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])
    #print(globals()[f"gs_results_Periode_{i}"].best_estimator_)
    
    globals()[f"grid_result_GBM_periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])

    # summarize results
    print("Periode " + str(i))
    print("Best: %f using %s" % (globals()[f"grid_result_GBM_periode_{i}"].best_score_, 
                                 globals()[f"grid_result_GBM_periode_{i}"].best_params_))
    globals()[f"means_periode_{i}"] = globals()[f"grid_result_GBM_periode_{i}"].cv_results_['mean_test_score']
    globals()[f"stds_periode_{i}"] = globals()[f"grid_result_GBM_periode_{i}"].cv_results_['std_test_score']
    globals()[f"params_periode_{i}"] = globals()[f"grid_result_GBM_periode_{i}"].cv_results_['params']
    for mean, stdev, param in zip(globals()[f"means_periode_{i}"], globals()[f"stds_periode_{i}"], globals()[f"params_periode_{i}"]):
        print("%f (%f) with: %r" % (mean, stdev, param))

Visualization of the results of the Grid-Search-Method for the Support Vector Method:

In [53]:
for i in Perioden:
    gs = GridSearchCV(
         estimator = estimator_SVM,
         param_grid = grid_params_SVM,
         verbose = 1,
         cv = 5,
         n_jobs = -1,
         scoring = 'accuracy'
         )

    globals()[f"grid_result_SVM_periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])

    # summarize results
    print("Periode " + str(i))
    print("Best: %f using %s" % (globals()[f"grid_result_SVM_periode_{i}"].best_score_, 
                                 globals()[f"grid_result_SVM_periode_{i}"].best_params_))
    globals()[f"means_periode_{i}"] = globals()[f"grid_result_SVM_periode_{i}"].cv_results_['mean_test_score']
    globals()[f"stds_periode_{i}"] = globals()[f"grid_result_SVM_periode_{i}"].cv_results_['std_test_score']
    globals()[f"params_periode_{i}"] = globals()[f"grid_result_SVM_periode_{i}"].cv_results_['params']
    for mean, stdev, param in zip(globals()[f"means_periode_{i}"], globals()[f"stds_periode_{i}"], globals()[f"params_periode_{i}"]):
        print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Periode 2009
Best: 0.501243 using {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.501243 (0.011131) with: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.499661 (0.011149) with: {'C': 0.3, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.498983 (0.010172) with: {'C': 0.5, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.499887 (0.009389) with: {'C': 1.0, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Periode 2011
Best: 0.511319 using {'C': 0.3, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.510309 (0.016815) with: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.511319 (0.019087) with: {'C': 0.3, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.510308 (0.018465) with: {'C': 0.5, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.509903 (0.018476) with: {'C': 1.0, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
Fitt

Visualization of the results of the Grid-Search-Method for Neural Networks:

In [43]:
for i in Perioden:

    # define baseline model
    def neural_model(neuron_hidden=10, activation_hidden='tanh', optimizer='adam', learning_rate=0.001, dropout=0.3, weight_decay=0.2):
        # create model
        model = Sequential()
        model.add(Dense(neuron_hidden, input_shape=(190,), activation=activation_hidden, use_bias=True, kernel_regularizer=regularizers.l1(weight_decay)))
        model.add(Dropout(dropout))
        model.add(Dense(3, activation='softmax', use_bias=True))
        model.summary()
        #optimizer1 = optimizer
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model 



    from keras.wrappers.scikit_learn import KerasClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold
    from sklearn.model_selection import GridSearchCV

    estimator = KerasClassifier(build_fn=neural_model, 
                                epochs=100, 
                                batch_size=300, 
                                verbose=3)



    grid = GridSearchCV(estimator=estimator, 
                        scoring= 'accuracy',
                        param_grid=grid_params_NN,
                        n_jobs=-1,
                        cv=5)

    
    globals()[f"grid_result_NN_periode_{i}"] = grid.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])

    # summarize results
    print("Periode " + str(i))
    print("Best: %f using %s" % (globals()[f"grid_result_NN_periode_{i}"].best_score_, 
                                 globals()[f"grid_result_NN_periode_{i}"].best_params_))
    globals()[f"means_periode_{i}"] = globals()[f"grid_result_NN_periode_{i}"].cv_results_['mean_test_score']
    globals()[f"stds_periode_{i}"] = globals()[f"grid_result_NN_periode_{i}"].cv_results_['std_test_score']
    globals()[f"params_periode_{i}"] = globals()[f"grid_result_NN_periode_{i}"].cv_results_['params']
    for mean, stdev, param in zip(globals()[f"means_periode_{i}"], globals()[f"stds_periode_{i}"], globals()[f"params_periode_{i}"]):
        print("%f (%f) with: %r" % (mean, stdev, param))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1910      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,943
Trainable params: 1,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 955       
                                                                 
 dropout_1 (Dropout)         (None, 5)                 0         
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 973
Trainable params: 973
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoc

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                1910      
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,943
Trainable params: 1,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                3820      
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_7 (Dense)             (None, 3)                 63        
                                                                 
Total params: 3,883
Trainable params: 3,883
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                3820      
                                                                 
 dropout_4 (Dropout)         (None, 20)                0         
                                                                 
 dense_9 (Dense)             (None, 3)                 63        
                                                                 
Total params: 3,883
Trainable params: 3,883
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 5)                 955       
                                                                 
 dropout_5 (Dropout)         (None, 5)                 0         
                                                                 
 dense_11 (Dense)            (None, 3)                 18        
                                                                 
Total params: 973
Trainable params: 973
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoc

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                1910      
                                                                 
 dropout_6 (Dropout)         (None, 10)                0         
                                                                 
 dense_13 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,943
Trainable params: 1,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100


In [41]:
grid_result_periode_2009.best_score_

0.5186440677966101

In [42]:
grid_result_periode_2009.best_params_

{'activation_hidden': 'tanh',
 'dropout': 0.3,
 'neuron_hidden': 5,
 'optimizer': 'adam',
 'weight_decay': 0.1}

Visualization of the results of the Grid-Search-Method for Random Forest:

In [47]:
for i in Perioden:
    gs = GridSearchCV(
         estimator = estimator_RF,
         param_grid = grid_params_RF,
         verbose = 1,
         cv = 5,
         n_jobs = -1,
         scoring = 'accuracy'
         )

    #print("Periode " + str(i))
    #globals()[f"gs_results_Periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])
    #print(globals()[f"gs_results_Periode_{i}"].best_estimator_)
    
    globals()[f"grid_result_RF_periode_{i}"] = gs.fit(globals()[f"x_train_periode_{i}"], globals()[f"y_train_periode_{i}"])

    # summarize results
    print("Periode " + str(i))
    print("Best: %f using %s" % (globals()[f"grid_result_RF_periode_{i}"].best_score_, 
                                 globals()[f"grid_result_RF_periode_{i}"].best_params_))
    globals()[f"means_periode_{i}"] = globals()[f"grid_result_RF_periode_{i}"].cv_results_['mean_test_score']
    globals()[f"stds_periode_{i}"] = globals()[f"grid_result_RF_periode_{i}"].cv_results_['std_test_score']
    globals()[f"params_periode_{i}"] = globals()[f"grid_result_RF_periode_{i}"].cv_results_['params']
    for mean, stdev, param in zip(globals()[f"means_periode_{i}"], globals()[f"stds_periode_{i}"], globals()[f"params_periode_{i}"]):
        print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Periode 2009
Best: 0.471412 using {'max_features': 'log2', 'min_samples_leaf': 2, 'n_estimators': 500}
0.465989 (0.053696) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 100}
0.468475 (0.053285) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 300}
0.461017 (0.066814) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 500}
0.465763 (0.062739) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 100}
0.464181 (0.062175) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 300}
0.465085 (0.058812) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 500}
0.468249 (0.059930) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 100}
0.470960 (0.054717) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 300}
0.466441 (0.058121) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n

Periode 2019
Best: 0.510094 using {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 300}
0.499718 (0.013186) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 100}
0.506257 (0.015135) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 300}
0.507962 (0.018753) with: {'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 500}
0.502135 (0.011508) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 100}
0.510094 (0.019443) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 300}
0.507109 (0.013082) with: {'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 500}
0.505262 (0.009503) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 100}
0.503982 (0.014702) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 300}
0.507679 (0.014312) with: {'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 500}
0.507251 (0.011640) with: {'max_features':